# Importing a CSV File Into a Postgres Database

## Steps:
- Import the CSV file into a pandas df
- Clean the table name and remove all extra symbols, spaces, capital letters
- Clean the column headers and remove all extra symbols, spaces, capital letters
- Write the create table SQL statement
- Import the data into the db

In [24]:
#import libraries
#conda install psycopg2
#pip install psycopg2

import os
import numpy as np
import pandas as pd
import psycopg2

# Find CSV Files in Directory

In [3]:
#Find CSV files in my current working directory
#Isolate only the CSV files
#Make a new directory
#Move the CSV files in the new directory

In [25]:
# Find CSV files in my current working directory
all_files = os.listdir(os.getcwd())
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
csv_files

['Customer Contracts$.csv', 'Customer Engagements.csv', 'Customer Demo.csv']

In [26]:
# Make a new directory
dataset_dir = 'dataset'

# Create the bash command to make a new directory
try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass
mkdir

'mkdir dataset'

In [27]:
# Move the CSV files in the new directory
for csv in csv_files:
    mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
    os.system(mv_file)
    print(mv_file)

mv 'Customer Contracts$.csv' dataset
mv 'Customer Engagements.csv' dataset
mv 'Customer Demo.csv' dataset


# Create the Pandas DF From the CSV File

In [28]:
data_path = os.getcwd()+'/'+dataset_dir+'/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
    print(file)

Customer Contracts$.csv
Customer Engagements.csv
Customer Demo.csv


In [29]:
df['Customer Contracts$.csv'].head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


# Clean Table Names and Column Names

In [30]:
for k in csv_files:
    dataframe = df[k]
    clean_tbl_name = k.lower().replace(" ","_").replace("?",""). \
    replace("-","_").replace(r"/","_").replace("\\","_") \
    .replace("%","").replace(")","").replace(r"(","").replace("$","")
    # print(clean_tbl_name)

    # Remove .csv extension from clean_tbl_name
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    print(tbl_name)

    dataframe.columns = [x.lower().replace(" ","_").replace("?",""). \
    replace("-","_").replace(r"/","_").replace("\\","_") \
    .replace("%","").replace(")","").replace(r"(","").replace("$","") for x in dataframe.columns]
    # print(dataframe.columns)

    # Replacement dictionary that maps pandas dtypes to sql dytpes
    replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'int',
    'datetime64' : 'timestamp',
    'timedelta64[ns]' : 'varchar'
    }
    replacements

    # Table Schema
    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    print(col_str)

    # Open a database connection
    conn_string = "host='localhost' dbname='pyautomation' user='root' password='secret' port='5432' options='-c search_path=dbo,public'"

    print ("Connecting to database\n ->%s" % (conn_string))

    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print("opened database successfully")

    # Drop tables with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    # Create table
    cursor.execute("create table %s (%s)" % (tbl_name, col_str))
    print('{0} was created succesfully'.format(tbl_name))

    # Insert Values to table

    # Save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    # Open the csv file, save it as an object
    my_file = open(k)
    print("File opened in memory")

    # Upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print("File copied to db")

    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()

    cursor.close()
    print("Table {0} to imported to db completed")

print('All tables have veen succesfully imported into db')

customer_contracts
customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar
Connecting to database
 ->host='localhost' dbname='pyautomation' user='root' password='secret' port='5432' options='-c search_path=dbo,public'
opened database successfully
customer_contracts was created succesfully
File opened in memory
File copied to db
Table {0} to imported to db completed
customer_engagements
customer_id int, num_of_users int, _of_all_employees varchar, sso varchar, launched varchar
Connecting to database
 ->host='localhost' dbname='pyautomation' user='root' password='secret' port='5432' options='-c search_path=dbo,public'
opened database successfully
customer_engagements was created succesfully
File opened in memory
File copied to db
Table {0} to imported to db completed
customer_demo
customer_id int, customer_name varchar, employee_count int, office_location varchar
Connecting to database
 ->host='localhost' dbname='pyautomati